# SW정글 week03 컴퓨팅 사고로의 전환
## <위상 정렬>
220418

In [23]:
# 2252 줄 세우기
from collections import deque
import sys
# input = sys.stdin.readline

N, M = map(int, input().split())

adj_list = [[] for _ in range(N+1)]
in_edges = [0]*(N+1)
for _ in range(M):
    A, B = map(int, input().split())
    adj_list[A].append(B)
    in_edges[B] += 1

res = []
queue = deque()
for node in range(1, N+1):
    if in_edges[node] == 0:
        queue.append(node)

while queue:
    now_node = queue.popleft()
    for adj_node in adj_list[now_node]:
        in_edges[adj_node] -= 1
        if in_edges[adj_node] == 0:
            queue.append(adj_node)
    res.append(now_node)

print(' '.join(map(str, res)))

In [34]:
# 2637 장난감 조립
from collections import deque
import sys
# input = sys.stdin.readline
N, M = int(input()), int(input())
adj_list = [{} for _ in range(N+1)]
in_degree = [0]*(N+1)
for _ in range(M):
    X, Y, K = map(int, input().split())
    adj_list[Y][X] = K
    in_degree[X] += 1

base_parts = []
for node in range(1, N+1):
    if in_degree[node] == 0:
        base_parts.append(node)

parts_info = [[0]*len(base_parts) for _ in range(N+1)]
for i in range(len(base_parts)):
    parts_info[base_parts[i]][i] = 1

queue = deque(base_parts)
while queue:
    now_part = queue.popleft()
    now_part_info = parts_info[now_part]
    for next_part, need_num in adj_list[now_part].items():
        next_part_info = parts_info[next_part]
        for i in range(len(base_parts)):
            next_part_info[i] += now_part_info[i]*need_num
        in_degree[next_part] -= 1
        if in_degree[next_part] == 0:
            queue.append(next_part)

for i in range(len(base_parts)):
    print(base_parts[i], parts_info[-1][i])

In [ ]:
# 1432 그래프 수정
from heapq import heappush, heappop
import sys
# input = sys.stdin.readline

n = int(input())
adj_list = [[] for _ in range((n+1))]
out_degree = [0] * (n+1)

for row in range(1, n+1):
    inp = input()
    for col in range(1, n+1):
        if inp[col-1] == '1':
            adj_list[col].append(row)
            out_degree[row] += 1

new_array = [0]*(n+1)

def topology_sort():
    queue = []
    for node in range(n, 0, -1):
        if out_degree[node] == 0:
            queue.append(-node) # out_degree가 0인(나가는 애만 있는) node를 node가 큰 게 먼저 들어오도록(후순위 배치)

    if not queue:
        print(-1)
        return

    new_num = n
    while queue:
        now_node = -heappop(queue) # queue에 들어 있는 애들 중에 node번호가 큰 애가 뒤로 가도록 -> queue에 들어 있는 애들끼리는 어떤 번호가 배정되든 상관 없다는 의미?! 근데 왜지...?
        # 큐에 들어온 애들의 의미 : 나보다 뒤에 있는 애들은 없다. 즉. 이제 내가 나와도 되는 상태다!! 그러니까, 큐에 들어 있는 애들은 언제든 나와도 되는 자유로운 애들만 들어오는 것이다.
        # 만약, a라는 노드의 뒤에 있는 b노드가 아직 queue에서 나오지 않았다면, a는 queue에 들어갈 자격이 없다.
        new_array[now_node] = new_num
        
        for adj_node in adj_list[now_node]:
            out_degree[adj_node] -= 1
            if out_degree[adj_node] == 0:
                heappush(queue, -adj_node)
            if out_degree[adj_node] == -1:
                print(-1)
                return
        new_num -= 1
    print(*new_array[1:])
    return

topology_sort()

In [1]:
# 1948 임계경로
from collections import deque
import sys
# input = sys.stdin.readline
n, m = int(input()), int(input())
in_degree = [0]*(n+1)
adj_list = [[] for _ in range(n+1)]

for _ in range(m):
    a, b, c = map(int, input().split())
    adj_list[a].append((b, c))
    in_degree[b] += 1

start, end = map(int, input().split())

queue = deque([start])
path_info = [[0, []] for _ in range(n+1)] # 각 노드를 거치는 path들 중 가장 크거나 같은 애들만 저장(cost, [prev_node_1, prev_node_2...])
path_info[start][1].append(None)
while queue:
    now_node = queue.popleft()
    cost = path_info[now_node][0]
    for adj_node, adj_cost in adj_list[now_node]:
        # 현재 노드(now_node)로 들어온 path들에 대해 인접 노드(adj_node)로 가는 경로 업데이트
        if path_info[adj_node][1]: # path가 들어 있으면
            adj_max_cost = path_info[adj_node][0]
            if cost+adj_cost == adj_max_cost: # 노드 추가
                path_info[adj_node][1].append(now_node)
            elif cost+adj_cost > adj_max_cost:
                path_info[adj_node][0] = cost + adj_cost
                path_info[adj_node][1] = [now_node]
        else: # path가 비어 있는 경우
            path_info[adj_node][0] = cost + adj_cost
            path_info[adj_node][1].append(now_node)
        
        in_degree[adj_node] -= 1    
        if in_degree[adj_node] == 0:
            queue.append(adj_node)

max_cost = path_info[end][0]

path_cnt = 0
visited = [0] * (n+1)
last_queue = deque([end])
while last_queue:
    now_node = last_queue.popleft()
    if now_node == start:
        continue
    for adj_node in path_info[now_node][1]:
        path_cnt += 1
        if adj_node is not None and not visited[adj_node]:
            last_queue.append(adj_node)
            visited[adj_node] = 1

print(max_cost)
print(path_cnt)

12
5
